In [1]:
!pip -q install grpcio grpcio-reflection


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
from langchain.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from caikit_nlp_client import HttpClient

# Embedding call:

### Set up inputs for vector conversion:

In [3]:
input_docs = [
    "Green is the color of envy, grass, and Kermit the frog.",
    "Red is the color of the planet Mars & the commonly associated with both Valentine's Day & Christmas holidays.",
    "Blue is the color of sadness & jazz music.",
    "Purple is the color you receive when mixing red & blue"
]

### Model/client config:

In [4]:
model_endpoint_url = "https://caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com"
embedding_model_name = "all-MiniLM-L12-v2-caikit"

http_client = HttpClient(model_endpoint_url, verify=False)
token = '''\
Bearer <token>
'''.replace('\n', '')

### Encode document chunks into vectors with embedding/bicoder model:

In [5]:
embedded_vectors = http_client.embedding_tasks(
    token=token,
    model_id=embedding_model_name,
    texts=input_docs,
)

input_to_vectors = []
for i in range(len(input_docs)):
    input_to_vectors.append({input_docs[i]: embedded_vectors["results"]["vectors"][i]})
print(json.dumps(input_to_vectors, sort_keys=True, indent=4))

Response: <Response [200]>
[
    {
        "Green is the color of envy, grass, and Kermit the frog.": {
            "data": {
                "values": [
                    0.029085179790854454,
                    0.06175018101930618,
                    0.09134913980960846,
                    -0.036822956055402756,
                    -0.015607893466949463,
                    0.08278024941682816,
                    0.1472027450799942,
                    -0.07463820278644562,
                    -0.016685882583260536,
                    0.0465996153652668,
                    -0.03764178976416588,
                    -0.06003865972161293,
                    -0.06265503168106079,
                    0.05606707185506821,
                    -0.0764918178319931,
                    0.045709121972322464,
                    0.05885414779186249,
                    -0.049785781651735306,
                    -0.06225161626935005,
                    -0.034113798290491104,
           

/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# Reranker / cross-encoder model call:

In [6]:
input_docs = [
        {
            "text":  "Green is the color of envy, grass, and Kermit the frog.",
            "title": "first title"
        },
        {
            "text": "Red is the color of the planet Mars & the commonly associated with both Valentine's Day & Christmas holidays.",
            "more": "more attributes here"
        },
        {
            "text": "Blue is the color of sadness & jazz music.",
            "meta": {"foo": "bar", "i": 999, "f": 12.34}
        },
        {
            "text": "Purple is the color you receive when mixing red & blue",
            "meta": "blah"
        }
    ]

### Pass list of docs to consider + a query, receive a sorted list of the docs by relevance to query

In [7]:
reranked_vectors = http_client.rerank(
    model_id=embedding_model_name,
    documents=input_docs,
    token=token,
    query="Tell me about the color red"
)

print(json.dumps(reranked_vectors, sort_keys=True, indent=4))

{
    "input_token_count": 75,
    "producer_id": {
        "name": "EmbeddingModule",
        "version": "0.0.1"
    },
    "result": {
        "query": "Tell me about the color red",
        "scores": [
            {
                "document": {
                    "more": "more attributes here",
                    "text": "Red is the color of the planet Mars & the commonly associated with both Valentine's Day & Christmas holidays."
                },
                "index": 1,
                "score": 0.5381495356559753,
                "text": "Red is the color of the planet Mars & the commonly associated with both Valentine's Day & Christmas holidays."
            },
            {
                "document": {
                    "meta": "blah",
                    "text": "Purple is the color you receive when mixing red & blue"
                },
                "index": 3,
                "score": 0.532557487487793,
                "text": "Purple is the color you receive whe

/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
